# Bank Statement Analysis 

Jupyter Notebook created to categorize .csv based bank statment expances.  

Currently suppoorted bank account export format is `ING Bank Śląski` which can aggreagate expances form other banks.

# Export data from Bank accounts


**ING Bank Śląski** -  Export the accounts you would like to do analysis for. Usually I'm esporting expances from all accounts except `Rachunki` bank account to see how much money was spenf to things like grocery. Exported file have `.csv` format and `cp-1250` encoding.

Data presnetation article
https://www.shanelynn.ie/data-visualisation-in-python-pycon-dublin-2018-presentation/




## Input Data

In [ ]:
ING_CSV = "data/mil_rach_2025.csv"

## Categorization

In [2]:
CATEGORY = {}

# 1. ING Bank Śląski



## 1.1 Import Data from ING Bank Sląski

Import raw data from .csv file.

`header` - is an index of non empty row in input csv file. Will be used as `df` column name.


In [44]:
import pandas as pd
from tabulate import tabulate

from helper import get_ing_header_row_index

# Dataframe print settings
pd.set_option('display.max_rows', None) # Show all rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.max_colwidth', None) # Optional: prevent truncating column width
pd.set_option('display.width', None) # Optional: wide display for Jupyter environments

# CSV file settings 
ING_ENCODING = "cp1250"
ING_SEP = ";"

ing_raw_df = pd.read_csv(
    ING_CSV,
    encoding=ING_ENCODING,
    sep=ING_SEP,
    header=get_ing_header_row_index(
        ING_CSV, ING_ENCODING
    ),
)

# Trim data frame - Drop last row which is empty
ing_raw_df.drop(ing_raw_df.tail(1).index, inplace=True)

## 1.2 - Data preparation

Ceanup the data from ING Bank account to extract only usefull columns

* Trim data fame to get only usefull columns for analysis
* Manipulate data
* Set data types


In [ ]:
# Drop columns with "Unnamed" in the header
df = ing_raw_df.loc[:, ~ing_raw_df.columns.str.startswith("Unnamed")]

# COPY Kwota blokady/zwolnienie blokady ->  Kwota transakcji (waluta rachunku)
df = df.copy() # needed due to error
df.loc[
    df["Kwota transakcji (waluta rachunku)"].isna(),
    "Kwota transakcji (waluta rachunku)"
] = df["Kwota blokady/zwolnienie blokady"]

# RENAME Kwota transakcji (waluta rachunku) -> Kwota transakcji
df = df.rename(columns={"Kwota transakcji (waluta rachunku)": "Kwota transakcji"})

# IF Dane kontrahenta empty COPY Tytuł
df["Dane kontrahenta"] = df.apply(
    lambda row: row["Tytuł"] if not row["Dane kontrahenta"].strip() else row["Dane kontrahenta"],
    axis=1
)

# Columns: Data transakcji; Dane kontrahenta; Kwota transakcji; 
df = df.copy()
df = df[["Data transakcji", "Dane kontrahenta", "Kwota transakcji"]]

# Add column
df["Kategoria"] = "NaN"


# *** SET data type ***

df["Data transakcji"] = df["Data transakcji"].astype("datetime64[ns]")
df["Dane kontrahenta"] = df["Dane kontrahenta"].astype(str)

df["Kwota transakcji"] = df["Kwota transakcji"].str.replace(",", ".").astype(float) # Replace comma with dot
df["Kwota transakcji"] = df["Kwota transakcji"].astype(float)

df["Kategoria"] = df["Kategoria"].astype(str)

df.dtypes

df.head(100)



,Data transakcji,Dane kontrahenta,Kwota transakcji,Kategoria
0,2025-04-19,UL. KOSCIUSZKI 44E GORLICE POL,-500.00,NaN
1,2025-04-19,SODEL ALBERT ŁUKASZ SADOWA 28 55-002 DOBRZYKOWICE,1.00,NaN
2,2025-04-19,SODEL ALBERT ŁUKASZ SADOWA 28 DOBRZYKOWICE,310.00,NaN
3,2025-04-19,Autopay Mobility Warszawa POL,-17.00,NaN
4,2025-04-19,Autopay Mobility Warszawa POL,-17.00,NaN
5,2025-04-19,ORLEN STACJA NR 459 LANY POL,-233.58,NaN
6,2025-04-19,LIDL STRACHOCINSKA WROCLAW POL,-164.20,NaN
7,2025-04-19,SODEL ALBERT ŁUKASZ SADOWA 28 DOBRZYKOWICE,600.00,NaN
8,2025-04-18,LIDL OLAWSKA JELCZ-LASKOWI POL 202 5-04-16,-138.98,NaN
9,2025-04-17,LIDL STRACHOCINSKA WROCLAW POL 202 5-04-15,-173.85,NaN


In [ ]:
# *** Trim data frame ***

"""
Fetch from raw data  frame only usefull columns, nothing more than that.
If needed first copy the data around to have coherent data set
"""

# Drop columns with "Unnamed" in the header
df = ing_raw_df.loc[:, ~ing_raw_df.columns.str.startswith("Unnamed")]




# *** Set Column Data types ***
"""
Might be usefull to analyse data afterwards.
"""

# ------------------------------------------------------------------------

# Set Data Types for columns
float_columns = [
    "Kwota transakcji (waluta rachunku)",
    "Kwota blokady/zwolnienie blokady",
    "Saldo po transakcji",
]

for col in float_columns:
    ing_raw_df[col] = ing_raw_df[col].str.replace(",", ".").astype(float)

# *** Manipulate data ***

#TODO: Why this is done like this?
# Set coherent column meaning for Millennium and ING by copying data from 'Tytuł' to 'Dane kontrahenta'
ing_raw_df["Dane kontrahenta"] = ing_raw_df.apply(
    lambda row: (
        row["Tytuł"]
        if row["Dane kontrahenta"] == "  "
        else row["Dane kontrahenta"]
    ),
    axis=1,
)
# from helper import print_df
# print_df(ing_raw_df)
# ing_raw_df

# TODO: Evalueate if we need to fill the missing values in the 'Data transakcji' column

# # If Data transakcji is  empty, fill it with Data księgowania
# df.loc[
#     df["Kwota transakcji (waluta rachunku)"].isna(),
#     "Kwota transakcji (waluta rachunku)",
# ] = df["Kwota blokady/zwolnienie blokady"]

# # For all blocked transactions fill the Kwota transakcji with tha blocked amount.
# df.loc[df["Data transakcji"].isna(), "Data transakcji"] = df["Data księgowania"]


## 1.3 ING Data analysis



In [19]:
pass